# Minimal PDF Chatbot with Gemini API
This notebook lets you upload a PDF and chat with it using Google's Gemini API. You can ask questions, request summaries, and more. The API key is loaded securely from a `.env` file.

## 1. Import Libraries and Load API Key

This cell imports all required libraries and loads your Gemini API key from the `.env` file.

In [11]:
import PyPDF2
import requests
from io import BytesIO
from IPython.display import display, Markdown
import ipywidgets as widgets
import os
from dotenv import load_dotenv

# Load Gemini API key from .env file
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY not found. Please set it in a .env file.")
GEMINI_API_URL = 'https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent'

## 2. Gemini API and PDF Extraction Helpers

This cell defines the function to call Gemini and a helper to extract text from PDFs.

In [12]:
def ask_gemini(prompt, context=None):
    """Send a prompt (and optional context) to Gemini API and return the response text."""
    headers = {
        'Content-Type': 'application/json',
        'X-goog-api-key': GEMINI_API_KEY
    }
    if context:
        full_prompt = f"Context: {context}\n\nUser: {prompt}"
    else:
        full_prompt = prompt
    data = {
        "contents": [{"parts": [{"text": full_prompt}]}]
    }
    response = requests.post(GEMINI_API_URL, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()['candidates'][0]['content']['parts'][0]['text']
    else:
        return f"Error: {response.status_code} - {response.text}"

def extract_pdf_text(pdf_bytes):
    """Extract all text from a PDF file (as bytes)."""
    reader = PyPDF2.PdfReader(BytesIO(pdf_bytes))
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

## 3. Upload PDF and Extract Text

Use the widget below to upload your PDF. The text will be extracted and stored for chat context.

In [13]:
upload_widget = widgets.FileUpload(accept='.pdf', multiple=False)
display(Markdown('**Upload a PDF to begin chatting:**'))
display(upload_widget)

pdf_text = ''

def on_upload_change(change):
    global pdf_text
    if upload_widget.value:
        file_info = upload_widget.value[0]
        pdf_bytes = file_info['content']
        pdf_text = extract_pdf_text(pdf_bytes)
        display(Markdown(f"**PDF loaded. Extracted {len(pdf_text)} characters.**"))
        display(Markdown('You can now ask questions or request a summary.'))

upload_widget.observe(on_upload_change, names='value')

**Upload a PDF to begin chatting:**

FileUpload(value=(), accept='.pdf', description='Upload')

**PDF loaded. Extracted 44550 characters.**

You can now ask questions or request a summary.

## 4. Chat with the PDF using Gemini

Type your questions or requests below. The chatbot will use the extracted PDF text as context for Gemini.

In [15]:
input_box = widgets.Textarea(placeholder='Ask a question about the PDF, or type "summarize"...', description='You:')
submit_btn = widgets.Button(description='Send', button_style='primary')
output_area = widgets.Output()

chat_history = []

def on_submit(_):
    user_query = input_box.value.strip()
    if not user_query:
        return
    if not pdf_text:
        with output_area:
            display(Markdown('**Please upload a PDF first.**'))
        return
    # Use the selected mode to modify the prompt
    mode = mode_dropdown.value
    if mode == 'Summarize':
        prompt = f"Summarize the following PDF content:\n{pdf_text}"
    elif mode == 'Extract keywords':
        prompt = f"Extract the main keywords from the following PDF content:\n{pdf_text}"
    else:
        prompt = f"{user_query}\n\nPDF Content:\n{pdf_text}"
    chat_history.append({'role': 'user', 'content': user_query})
    with output_area:
        display(Markdown(f'**You:** {user_query}'))
        display(Markdown('**Gemini is thinking...**'))
    # Call Gemini API
    response = ask_gemini(prompt)
    chat_history.append({'role': 'gemini', 'content': response})
    with output_area:
        display(Markdown(f'**Gemini:** {response}'))
    input_box.value = ''
    output_area.clear_output(wait=True)
    # Re-display chat history
    for turn in chat_history:
        if turn['role'] == 'user':
            display(Markdown(f'**You:** {turn["content"]}'))
        else:
            display(Markdown(f'**Gemini:** {turn["content"]}'))

submit_btn.on_click(on_submit)
display(input_box, submit_btn, output_area)

Textarea(value='', description='You:', placeholder='Ask a question about the PDF, or type "summarize"...')

Button(button_style='primary', description='Send', style=ButtonStyle())

Output()

## 5. Extra Features

The following cells add preview, chat clearing, mode selection, token count, and chat download features.

In [16]:
# Preview the first 1000 characters of the extracted PDF text
def preview_pdf_text():
    if pdf_text:
        display(Markdown(f"**Preview of extracted PDF text (first 1000 chars, total {len(pdf_text)} chars):**"))
        display(Markdown(f'<pre style="max-height:200px;overflow:auto">{pdf_text[:1000]}</pre>'))
    else:
        display(Markdown('**No PDF text extracted yet.**'))
preview_pdf_text()

**Preview of extracted PDF text (first 1000 chars, total 44550 chars):**

<pre style="max-height:200px;overflow:auto"> 
I.
 
Outline
 
of
 
the
 
Research
 
Project
 
I(i)
 
Executive
 
Summary
 
of
 
the
 
Research
 
Project
 
(Include
 
the
 
problem
 
statement,
 
research
 
aims/goals,
 
and
 
methodology.
 
Maximum
 
500
 
words.)
 
Local
 
communication
 
systems
 
are
 
frequently
 
destroyed
 
by
 
natural
 
disasters,
 
leaving
 
survivors
 
alone
 
and
 
postponing
 
rescue
 
efforts.
 
Traditional
 
solutions,
 
like
 
satellite
 
beacons
 
or
 
Bluetooth
 
trackers,
 
are
 
insufficient
 
because
 
emergency
 
beacons
 
are
 
expensive,
 
power-hungry,
 
and
 
not
 
suitable
 
for
 
widespread
 
deployment,
 
while
 
consumer
 
trackers
 
depend
 
on
 
networks
 
that
 
aren't
 
reliable
 
in
 
emergency
 
situations.This
 
creates
 
a
 
critical
 
gap
 
for
 
an
 
affordable,
 
long-lasting,
 
and
 
resilient
 
communication
 
system.
 
This
 
project
 
introduces
 
L.A.S.T.
 
(Last-mile
 
Aid
 
&
 
Survival
 
Transmission),
 
a
 
disaster-ready
 
framework
 
designed
 
to
 
meet
 
the
 
</pre>

In [17]:
# Chat mode selection and clear chat button
mode_dropdown = widgets.Dropdown(options=['Ask a question', 'Summarize', 'Extract keywords'], value='Ask a question', description='Mode:')
clear_btn = widgets.Button(description='Clear Chat', button_style='danger')

def clear_chat(_):
    global chat_history
    chat_history = []
    output_area.clear_output()
    display(Markdown('**Chat history cleared.**'))

clear_btn.on_click(clear_chat)
display(mode_dropdown, clear_btn)

Dropdown(description='Mode:', options=('Ask a question', 'Summarize', 'Extract keywords'), value='Ask a questi…

Button(button_style='danger', description='Clear Chat', style=ButtonStyle())

In [18]:
# Show number of characters and rough token estimate sent to Gemini
def show_token_info():
    if pdf_text:
        char_count = len(pdf_text)
        # Rough estimate: 1 token ≈ 4 chars (varies by language/model)
        token_est = char_count // 4
        display(Markdown(f"**PDF context: {char_count} characters (~{token_est} tokens) will be sent to Gemini.**"))
    else:
        display(Markdown('**No PDF text extracted yet.**'))
show_token_info()

**PDF context: 44550 characters (~11137 tokens) will be sent to Gemini.**

In [19]:
# Download chat history as a text file
import io
from IPython.display import FileLink

def download_chat_history():
    if not chat_history:
        display(Markdown('**No chat history to download.**'))
        return
    lines = []
    for turn in chat_history:
        role = 'You' if turn['role'] == 'user' else 'Gemini'
        lines.append(f'{role}: {turn["content"]}\n')
    content = '\n'.join(lines)
    with open('chat_history.txt', 'w', encoding='utf-8') as f:
        f.write(content)
    display(FileLink('chat_history.txt'))

download_btn = widgets.Button(description='Download Chat History', button_style='info')
download_btn.on_click(lambda _: download_chat_history())
display(download_btn)

Button(button_style='info', description='Download Chat History', style=ButtonStyle())